In [119]:
import requests
from time import sleep
from datetime import date
import pandas as pd

In [120]:
newsrooms = pd.read_csv('../data/raw/newsrooms.csv')
newsrooms.columns = ['site', 'monthly_visits', 'country', 'country_of_pub']
newsrooms.dropna(inplace=True)
newsrooms = newsrooms.drop_duplicates(subset = 'site')
newsrooms = newsrooms.sort_values(by=['country_of_pub', 'monthly_visits'], ascending=False).reset_index().drop('index', axis=1)

In [121]:
UK = newsrooms[newsrooms['country_of_pub']=='UK'].reset_index().drop('index', axis=1)
USA = newsrooms[newsrooms['country_of_pub']=='USA'].reset_index().drop('index', axis=1)
India = newsrooms[newsrooms['country_of_pub']=='India'].reset_index().drop('index', axis=1)
SA = newsrooms[newsrooms['country_of_pub']=='South Africa'].reset_index().drop('index', axis=1)

In [184]:
all_sites = list(SA.site.unique().flatten()) + list(USA.site.unique().flatten()) + list(UK.site.unique().flatten()) + list(India.site.unique().flatten())

In [186]:
len(all_sites)

203

In [137]:
len(UK) + len(USA) + len(India) + len(SA)

203

In [165]:
year = 2020

def get_two_month_ranges(year):
    month_ranges = [[[1, 1, year], [31, 1, year]], 
                    [[1, 2, year], [28, 2, year]],
                    [[1, 3, year], [31, 3, year]],
                    [[1, 4, year], [30, 4, year]],
                    [[1, 5, year], [31, 5, year]],
                    [[1, 6, year], [30, 6, year]],
                    [[1, 7, year], [31, 7, year]],
                    [[1, 8, year], [31, 8, year]],
                    [[1, 9, year], [30, 9, year]],
                    [[1, 10, year], [31, 10, year]],
                    [[1, 11, year], [30, 11, year]],
                    [[1, 12, year], [31, 12, year]]]

    two_month_ranges = []
    for i in range(len(month_ranges)-1):
        if i%2==0:
            two_month_ranges.append((month_ranges[i][0], month_ranges[i+1][1]))

    return two_month_ranges

In [202]:
month_ranges = []
for i in range(2019,2020):
    month_ranges = month_ranges + get_two_month_ranges(i)

In [203]:
month_ranges

[([1, 1, 2019], [28, 2, 2019]),
 ([1, 3, 2019], [30, 4, 2019]),
 ([1, 5, 2019], [30, 6, 2019]),
 ([1, 7, 2019], [31, 8, 2019]),
 ([1, 9, 2019], [31, 10, 2019]),
 ([1, 11, 2019], [31, 12, 2019])]

In [187]:
import tqdm

In [215]:
%%time

from tqdm import tqdm
headlines_dict = {}

sites = all_sites

url = "https://google-news.p.rapidapi.com/v1/source_search"

for i in tqdm(month_ranges):
    from_date = f'{i[0][2]}-{i[0][1]}-{i[0][0]}'
    to_date = f'{i[1][2]}-{i[1][1]}-{i[1][0]}'
    year = i[0][2]
    bimester = i[1][1]/2

    for site in sites:

        print("Scraping" ,site, "starting from", from_date, "to", to_date)

        urls = []
        headlines = []
        times = []
        scrape_dates = []
        websites = []

        # baseline news

        #querystring = { "lang":"en","from":from_date,"to":to_date,"source":site}

        # women's news

        querystring = {"q": "women | woman | girl  | female  | lady | ladies | she | her | herself | aunt | grandmother | mother | sister | daughter | wife | mom | mum | girlfriend | mrs | niece " , "lang":"en","from":from_date,"to":to_date,"source":site}

        headers = {
            # Paid API
            'x-rapidapi-key': "bd5010b7eemsh00ab5684fa6a7efp15b791jsnd00e2d098829",

            # Free API
            #'x-rapidapi-key': "3e81132ebdmshafd93b150db164ep13c18fjsn834550268d59",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }

        response = requests.request("GET", url, headers=headers, params=querystring).json()

        request_count += 1

        for j in range(len(response['articles'])):
            headlines.append(response['articles'][j]['title'])
            times.append(response['articles'][j]['published'])
            urls.append(response['articles'][j]['link'])
            websites.append(response['articles'][j]['source']['href'])

        headlines_dict = {"url": urls, "headline": headlines, "time": times, "scrape_date": date.today().strftime("%d/%m/%Y"), "site": site}

        headlines_df = pd.DataFrame.from_dict(headlines_dict)
        number_of_articles = len(headlines_df)
        # save results to csv

        headlines_df.to_csv(f'../data/raw_temporal/{number_of_articles}_{site}_{bimester}_{year}_articles.csv')
        # print final statement
        print("Saved data from " + str(number_of_articles) + " articles of "  + str(site) + " to .csv!")
        print(request_count, "requests fired")




com starting from 2019-11-1 to 2019-12-31
Saved data from 87 articles of thedailybeast.com to .csv!
1280 requests fired
Scraping realclearpolitics.com starting from 2019-11-1 to 2019-12-31
Saved data from 0 articles of realclearpolitics.com to .csv!
1281 requests fired
Scraping cbssports.com starting from 2019-11-1 to 2019-12-31
Saved data from 13 articles of cbssports.com to .csv!
1282 requests fired
Scraping yahoo.com starting from 2019-11-1 to 2019-12-31
Saved data from 100 articles of yahoo.com to .csv!
1283 requests fired
Scraping lifehacker.com starting from 2019-11-1 to 2019-12-31
Saved data from 2 articles of lifehacker.com to .csv!
1284 requests fired
Scraping buzzfeed.com starting from 2019-11-1 to 2019-12-31
Saved data from 81 articles of buzzfeed.com to .csv!
1285 requests fired
Scraping nationalgeographic.com starting from 2019-11-1 to 2019-12-31
Saved data from 1 articles of nationalgeographic.com to .csv!
1286 requests fired
Scraping reuters.com starting from 2019-11-1 t

In [174]:
response

{'status': 'error',
 'error_code': 'UnsupportedParameter',
 'message': "Found unsupported parameter(s)['source']"}